# Re adaptation of the ABC location Policy for primarily classifying the Mulimodality conditions in Milan's PUMPS Area.

### the model runs in a parts, so you need to run each model, restart Kernal before moving to the other one.

In [1]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd

In [2]:
# Load the shapefiles
train_gdf = gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\train_stops_PUMS.shp").to_crs(epsg=32632)
metro_gdf = gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\metro_stops_PUMS.shp").to_crs(epsg=32632)
bus_gdf = gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\bus_90.shp").to_crs(epsg=32632)
tram_gdf = gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\tram_pumS.shp").to_crs(epsg=32632)
parking_gdf=gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD Readings\ABC\B\P+R.shp").to_crs(epsg=32632)
reg_bus_gdf=gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\reg-bus.shp").to_crs(epsg=32632)
bus_tranne_90= gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\bus_not_90.shp").to_crs(epsg=32632)
highway_exit= gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\highway_exit.shp").to_crs(epsg=32632)
parking= gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\parking_lot.shp").to_crs(epsg=32632)
#bus_others_gdf=gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-22-01-2024\bus-others.shp").to_crs(epsg=32632)


In [3]:
# Adding a column 'int_scr' with the specified values for each DataFrame
train_gdf['int_scr'] = 6
metro_gdf['int_scr'] = 5
tram_gdf['int_scr'] = 4
bus_gdf['int_scr'] = 3
bus_tranne_90['int_scr'] = 3
reg_bus_gdf['int_scr'] = 2
parking_gdf['int_scr'] = 1
highway_exit['int_scr'] = 1
parking['int_scr'] = 1

## Train_ABC-model

In [ ]:
# Merge the point layers into a single GeoDataFrame
points_gdf = gpd.GeoDataFrame(pd.concat([metro_gdf, bus_gdf, tram_gdf,parking_gdf,reg_bus_gdf,bus_tranne_90,highway_exit,parking], ignore_index=True))

In [ ]:
# Create a 125-meter buffer around the Train points
buffer_distance = 125
train_gdf['buffer'] = train_gdf.geometry.buffer(buffer_distance)

In [ ]:
train_scores = {}

In [ ]:
# Perform the intersection and calculate the sum of unique 'int_scr' values for each buffered train feature
for index, train_feature in train_gdf.iterrows():
    buffer_zone = train_feature['buffer']
    # Find points within the buffer zone
    intersected_points = points_gdf[points_gdf.geometry.intersects(buffer_zone)]
    # Sum the unique 'int_scr' values
    unique_scores = set(intersected_points['int_scr'].tolist())
    # Update train_gdf with the unique sum score for each train feature
    train_gdf.at[index, 'unique_sum'] = sum(unique_scores)

# Now train_gdf has an additional column 'unique_sum' with the calculated scores

In [ ]:
train_gdf.at[index,'unique_sum'] = int(sum(unique_scores))
train_gdf['scr_abc']= train_gdf['unique_sum'] + train_gdf ['int_scr']
#train_gdf

In [ ]:
train_gdf.to_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\train_scr.csv")

## METRO ABC MODEL

In [ ]:
# Merge the point layers into a single GeoDataFrame
per_metro_gdf = gpd.GeoDataFrame(pd.concat([train_gdf, bus_gdf, tram_gdf,parking_gdf,reg_bus_gdf,bus_tranne_90,highway_exit,parking], ignore_index=True))

In [ ]:
# Create a 125-meter buffer around the Train points
buffer_distance = 125
metro_gdf['buffer'] = metro_gdf.geometry.buffer(buffer_distance)

In [ ]:
metro_scores = {}

In [ ]:
# Perform the intersection and calculate the sum of unique 'int_scr' values for each buffered train feature
for index, metro_feature in metro_gdf.iterrows():
    buffer_zone = metro_feature['buffer']
    # Find points within the buffer zone
    intersected_points = per_metro_gdf[per_metro_gdf.geometry.intersects(buffer_zone)]
    # Sum the unique 'int_scr' values
    unique_scores = set(intersected_points['int_scr'].tolist())
    metro_scores[metro_feature['stop_id']] = sum(unique_scores)

In [ ]:
# Perform the intersection and calculate the sum of unique 'int_scr' values for each buffered train feature
for index, metro_feature in metro_gdf.iterrows():
    buffer_zone = metro_feature['buffer']
    # Find points within the buffer zone
    intersected_points = per_metro_gdf[per_metro_gdf.geometry.intersects(buffer_zone)]
    # Sum the unique 'int_scr' values
    unique_scores = set(intersected_points['int_scr'].tolist())
    # Update train_gdf with the unique sum score for each train feature
    metro_gdf.at[index, 'unique_sum'] = sum(unique_scores)

# Now metro_gdf has an additional column 'unique_sum' with the calculated scores


In [ ]:
metro_gdf.at[index, 'unique_sum'] = int(sum(unique_scores))
metro_gdf['scr_abc']= metro_gdf['unique_sum'] + metro_gdf ['int_scr']
metro_gdf

In [ ]:
metro_gdf.to_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\metro_scr.csv")

## TRAM_ABC_MODEL

In [4]:
# Merge the point layers into a single GeoDataFrame
per_tram_gdf = gpd.GeoDataFrame(pd.concat([metro_gdf, bus_gdf, train_gdf,parking_gdf,reg_bus_gdf,bus_tranne_90,highway_exit,parking], ignore_index=True))

In [5]:
# Create a 125-meter buffer around the Train points
buffer_distance = 125
tram_gdf['buffer'] = tram_gdf.geometry.buffer(buffer_distance)

In [6]:
tram_gdf

,field_1,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,pums,s_lat,s_lon,geometry,int_scr,buffer
0,1.0,14827.0,15_11856,P.ta Venezia M1,P.ta Venezia M1,45.474501,9.205612,no,516071.0176,5035684.195,POINT (516071.018 5035684.195),4,"POLYGON ((516196.018 5035684.195, 516195.416 5..."
1,2.0,14828.0,15_11859,P.le Principessa Clotilde (Osp. Fatebenefratelli),P.le Principessa Clotilde (Osp. Fatebenefratelli),45.480406,9.192552,no,515048.6503,5036337.653,POINT (515048.650 5036337.653),4,"POLYGON ((515173.650 5036337.653, 515173.048 5..."
2,9.0,14838.0,15_15049,P.le Cim.Monumentale Via Bramante,P.le Cim.Monumentale Via Bramante,45.483715,9.177737,no,513890.0300,5036702.669,POINT (513890.030 5036702.670),4,"POLYGON ((514015.030 5036702.670, 514014.428 5..."
3,12.0,14842.0,15_12620,"V.le Ungheria, 20","V.le Ungheria, 20",45.445864,9.246270,no,519258.6787,5032511.627,POINT (519258.679 5032511.627),4,"POLYGON ((519383.679 5032511.627, 519383.077 5..."
4,15.0,14846.0,15_11323,Cenisio M5,Cenisio M5,45.487559,9.172963,no,513516.0278,5037128.851,POINT (513516.028 5037128.851),4,"POLYGON ((513641.028 5037128.851, 513640.426 5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784,NaN,NaN,None,None,None,NaN,NaN,yes,518982.4610,5031794.983,POINT (518982.461 5031794.983),4,"POLYGON ((519107.461 5031794.983, 519106.859 5..."
785,NaN,NaN,None,None,None,NaN,NaN,yes,519447.2682,5031958.369,POINT (519447.268 5031958.370),4,"POLYGON ((519572.268 5031958.370, 519571.666 5..."
786,NaN,NaN,None,None,None,NaN,NaN,yes,519558.5402,5032082.318,POINT (519558.540 5032082.319),4,"POLYGON ((519683.540 5032082.319, 519682.938 5..."
787,NaN,NaN,None,None,None,NaN,NaN,yes,519916.3008,5032421.768,POINT (519916.301 5032421.769),4,"POLYGON ((520041.301 5032421.769, 520040.699 5..."


tram_scores = {}

In [7]:
# Perform the intersection and calculate the sum of unique 'int_scr' values for each buffered train feature
for index, tram_feature in tram_gdf.iterrows():
    buffer_zone = tram_feature['buffer']
    # Find points within the buffer zone
    intersected_points = per_tram_gdf[per_tram_gdf.geometry.intersects(buffer_zone)]
    # Sum the unique 'int_scr' values
    unique_scores = set(intersected_points['int_scr'].tolist())
    tram_scores[tram_feature['stop_id']] = sum(unique_scores)

NameError: name 'tram_scores' is not defined

In [ ]:
# Perform the intersection and calculate the sum of unique 'int_scr' values for each buffered train feature
for index, tram_feature in tram_gdf.iterrows():
    buffer_zone = tram_feature['buffer']
    # Find points within the buffer zone
    intersected_points = per_tram_gdf[per_tram_gdf.geometry.intersects(buffer_zone)]
    # Sum the unique 'int_scr' values
    unique_scores = set(intersected_points['int_scr'].tolist())
    # Update train_gdf with the unique sum score for each train feature
    tram_gdf.at[index, 'unique_sum'] = sum(unique_scores)

# Now tram_gdf has an additional column 'unique_sum' with the calculated scores


In [ ]:
tram_gdf.at[index, 'unique_sum'] = int(sum(unique_scores))
tram_gdf['scr_abc']= tram_gdf['unique_sum'] + tram_gdf['int_scr']
tram_gdf

In [ ]:
tram_gdf.to_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\tram_scr.csv")

## BUS_ABC_MODEL

In [ ]:
# Merge the point layers into a single GeoDataFrame
per_bus_gdf = gpd.GeoDataFrame(pd.concat([metro_gdf, train_gdf, tram_gdf,parking_gdf,reg_bus_gdf,bus_tranne_90,highway_exit,parking], ignore_index=True))

In [ ]:
# Create a 125-meter buffer around the Train points
buffer_distance = 125
bus_gdf['buffer'] = bus_gdf.geometry.buffer(buffer_distance)

In [ ]:
bus_scores = {}

In [ ]:
# Perform the intersection and calculate the sum of unique 'int_scr' values for each buffered train feature
for index, bus_feature in bus_gdf.iterrows():
    buffer_zone = bus_feature['buffer']
    # Find points within the buffer zone
    intersected_points = per_bus_gdf[per_bus_gdf.geometry.intersects(buffer_zone)]
    # Sum the unique 'int_scr' values
    unique_scores = set(intersected_points['int_scr'].tolist())
    bus_scores[bus_feature['stop_id']] = sum(unique_scores)

In [ ]:
# Perform the intersection and calculate the sum of unique 'int_scr' values for each buffered train feature
for index, bus_feature in bus_gdf.iterrows():
    buffer_zone = bus_feature['buffer']
    # Find points within the buffer zone
    intersected_points = per_bus_gdf[per_bus_gdf.geometry.intersects(buffer_zone)]
    # Sum the unique 'int_scr' values
    unique_scores = set(intersected_points['int_scr'].tolist())
    # Update train_gdf with the unique sum score for each train feature
    bus_gdf.at[index, 'unique_sum'] = sum(unique_scores)

# Now tram_gdf has an additional column 'unique_sum' with the calculated scores

In [ ]:
bus_gdf.at[index, 'unique_sum'] = int(sum(unique_scores))
bus_gdf['scr_abc']= bus_gdf['unique_sum'] + bus_gdf['int_scr']
bus_gdf

In [ ]:
bus_gdf.to_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\bus_scr.csv")